In [1]:
import numpy as np
from hw6_dataload import LFD_Data

# HW 8
## Primal vs Dual Problem

- The SVM **primal** problem minimizes 0.5w<sup>T</sup>w subject to y<sub>n</sub>(w<sup>T</sup>x<sub>n</sub>+b)&ge;1 for n = 1,2,...,N
- Since w is a d-dimensional vector (corresponding to the dimension of x) and we can also vary b, the primal problem involves a quadratic programming problem with **d+1 variables**.

## Polynomial Kernels

- Implementing polynimal kernels with a soft-margin SVM using the given data set of handwritten digits from the US Postal Service Zip Code data set with extracted features digit, intensity, and symmetry. 
- The polynomial kernel K(x<sub>n</sub>, x<sub>m</sub>) = (1+x<sub>n</sub><sup>T</sup>x<sub>m</sub>)<sup>Q</sup>
- Training **two** types of binary classifiers:
    - one-vs-one (one digit class is +1, another is -1, rest are ignored)
    - one-vs-all (one digit class is +1, everything else is -1)